# SQL Practice

The dataset and exercises are from [kiwidamien repository](https://github.com/kiwidamien/SQL_practice)

In [3]:
import pandas as pd
from pandasql import sqldf
import sqlite3
import math

In [4]:
pysqldf = lambda q: sqldf(q, globals())

## 1. Seattle Weather

In [6]:
weather = pd.read_csv("01_seattle_weather/cleaned_weather.csv")
conn_1 = sqlite3.connect(":memory:")
weather.to_sql('weather', conn_1, index=False)


25548

**1. Select all rows from December 1st, 2000 to December 15th, 2000 (inclusive)**

In [8]:
query = """
SELECT *
FROM weather
WHERE DATE >= "2000-12-01"
AND DATE <= "2000-12-15"
"""

result =  pd.read_sql_query(query, conn_1)
print(result)

          DATE  PRCP  TMAX  TMIN  RAIN
0   2000-12-01  0.04    55    39     1
1   2000-12-02  0.18    51    37     1
2   2000-12-03  0.00    44    34     0
3   2000-12-04  0.00    51    37     0
4   2000-12-05  0.00    50    36     0
5   2000-12-06  0.00    50    35     0
6   2000-12-07  0.00    40    34     0
7   2000-12-08  0.02    45    30     1
8   2000-12-09  0.06    43    36     1
9   2000-12-10  0.00    40    30     0
10  2000-12-11  0.00    37    28     0
11  2000-12-12  0.00    37    28     0
12  2000-12-13  0.00    37    30     0
13  2000-12-14  0.26    46    28     1
14  2000-12-15  0.00    42    35     0


**2. Get the average maximum temperature for every year from the year 2000 onward. Order the results by year (ascending)**

In [10]:
query = """
SELECT STRFTIME('%Y', DATE) AS YEAR,
AVG(TMAX)
FROM weather
GROUP BY YEAR
HAVING YEAR >= "2000"
"""

result =  pd.read_sql_query(query, conn_1)
print(result)

    YEAR  AVG(TMAX)
0   2000  58.674863
1   2001  58.473973
2   2002  58.893151
3   2003  60.441096
4   2004  60.622951
5   2005  60.148352
6   2006  61.038356
7   2007  59.202740
8   2008  58.494536
9   2009  59.912329
10  2010  59.663014
11  2011  58.139726
12  2012  59.502732
13  2013  60.901370
14  2014  62.594521
15  2015  63.369863
16  2016  62.546448
17  2017  61.727011


**3 Get the standard deviation of the maximum temperature per year, from 2000 onward. Order by year (ascending)**

In [12]:
conn_1.create_function("SQRT", 1, math.sqrt)

In [13]:
query = """
SELECT 
    STRFTIME('%Y', DATE) AS YEAR,
    AVG(TMAX) AS mean_TMAX,
    AVG(TMAX * TMAX) - AVG(TMAX) * AVG(TMAX) AS variance_TMAX,
    SQRT(AVG(TMAX * TMAX) - AVG(TMAX) * AVG(TMAX)) AS standard_deviation_TMAX
FROM weather
GROUP BY YEAR
HAVING YEAR >= "2000"
"""

result =  pd.read_sql_query(query, conn_1)
print(result)

    YEAR  mean_TMAX  variance_TMAX  standard_deviation_TMAX
0   2000  58.674863     131.574614                11.470598
1   2001  58.473973     124.545213                11.159983
2   2002  58.893151     151.026939                12.289302
3   2003  60.441096     165.238311                12.854505
4   2004  60.622951     158.606468                12.593906
5   2005  60.148352     141.054915                11.876654
6   2006  61.038356     169.790310                13.030361
7   2007  59.202740     166.391773                12.899294
8   2008  58.494536     168.463085                12.979333
9   2009  59.912329     202.019711                14.213364
10  2010  59.663014     124.196029                11.144327
11  2011  58.139726     154.635271                12.435243
12  2012  59.502732     161.954911                12.726151
13  2013  60.901370     184.516299                13.583678
14  2014  62.594521     170.723258                13.066111
15  2015  63.369863     173.285119      

**4. What are the 10 hottest days on record? Take hottest to mean 'highest maximum temperature'**

In [15]:
query = """
SELECT *
FROM weather
ORDER BY TMAX DESC
LIMIT 10
"""

result =  pd.read_sql_query(query, conn_1)
print(result)

         DATE  PRCP  TMAX  TMIN  RAIN
0  2009-07-29   0.0   103    71     0
1  1994-07-20   0.0   100    65     0
2  1960-08-09   0.0    99    59     0
3  1981-08-09   0.0    99    68     0
4  1991-07-23   0.0    99    65     0
5  1960-08-08   0.0    98    66     0
6  1967-08-16   0.0    98    59     0
7  1979-07-16   0.0    98    63     0
8  1981-08-10   0.0    98    67     0
9  1988-09-02   0.0    98    59     0


**5. In 2016, what fraction of days did it rain?**

In [17]:
query = """
SELECT AVG(RAIN) AS rain_fraction_2016
FROM weather
WHERE DATE >= "2016-01-01"
AND DATE <= "2016-12-31"
"""

result =  pd.read_sql_query(query, conn_1)
print(result)

   rain_fraction_2016
0            0.469945


**6. What is the 75th percentile for the amount of rain that fell on a day where there was some rain in 2016?**

In [19]:
query = """
SELECT PRCP
FROM (
    SELECT PRCP
    FROM weather
    WHERE 
        PRCP > 0 AND
        STRFTIME('%Y', DATE) = '2016'
    ORDER BY PRCP
    LIMIT 1 OFFSET (
        (SELECT COUNT(*) * 0.75 
         FROM weather
         WHERE PRCP > 0 AND STRFTIME('%Y', DATE) = '2016')
    )
) AS sub
"""

result =  pd.read_sql_query(query, conn_1)
print(result)

   PRCP
0  0.33


**7. What is the 75th percentile for the amount of rain that fell on any day in 2016?**

In [21]:
query = """
SELECT PRCP
    FROM weather
    WHERE 
        STRFTIME('%Y', DATE) = '2016'
    ORDER BY PRCP
    LIMIT 1 OFFSET ROUND((SELECT COUNT(*) * 0.75 
         FROM weather
         WHERE STRFTIME('%Y', DATE) = '2016'))
    
"""

result =  pd.read_sql_query(query, conn_1)
print(result)

   PRCP
0  0.15


**8. Get the 10 years with the hottest average maximum temperature in July. Order from hottest to coolest**

In [23]:
query = """
SELECT 
    STRFTIME('%Y', DATE) AS YEAR,
    AVG(TMAX) AS avg_july_high_temp
FROM weather   
WHERE STRFTIME('%m', DATE) = "07"
GROUP BY YEAR
ORDER BY avg_july_high_temp DESC
LIMIT 10
"""

result =  pd.read_sql_query(query, conn_1)
print(result)

   YEAR  avg_july_high_temp
0  2015           82.580645
1  1958           81.419355
2  2009           80.967742
3  1985           80.935484
4  2014           80.419355
5  1960           79.645161
6  1965           79.451613
7  1990           79.193548
8  2013           78.967742
9  2003           78.967742


**9. Get the 10 years with the coldest average minimum temperature in December. Order from coolest to hottest**

In [25]:
query = """
SELECT 
    STRFTIME('%Y', DATE) AS YEAR,
    AVG(TMIN) AS avg_dec_min_temp
FROM weather   
WHERE STRFTIME('%m', DATE) = "12"
GROUP BY YEAR
ORDER BY avg_dec_min_temp ASC
LIMIT 10
"""

result =  pd.read_sql_query(query, conn_1)
print(result)

   YEAR  avg_dec_min_temp
0  1990         30.387097
1  1948         30.806452
2  1985         30.935484
3  1951         31.225806
4  1964         31.483871
5  1983         31.516129
6  1968         32.032258
7  1984         32.096774
8  2009         32.096774
9  1978         32.161290


**10. Repeat the last question, but round the temperatures to 3 decimal places**

In [27]:
query = """
SELECT 
    STRFTIME('%Y', DATE) AS YEAR,
    ROUND(AVG(TMIN), 3) AS avg_dec_min_temp
FROM weather   
WHERE STRFTIME('%m', DATE) = "12"
GROUP BY YEAR
ORDER BY avg_dec_min_temp ASC
LIMIT 10
"""

result =  pd.read_sql_query(query, conn_1)
print(result)

   YEAR  avg_dec_min_temp
0  1990            30.387
1  1948            30.806
2  1985            30.935
3  1951            31.226
4  1964            31.484
5  1983            31.516
6  1968            32.032
7  1984            32.097
8  2009            32.097
9  1978            32.161


**11. Give the average inches of rain that fell per day for each month, where the average is taken over 2000 - 2010 (inclusive)**

In [29]:
query = """
SELECT
    STRFTIME('%m', DATE) AS month,
    AVG(PRCP) as avg_daily_inches_rain
FROM weather
WHERE STRFTIME('%Y', DATE) >= "2000" AND STRFTIME('%Y', DATE) <= "2010"
GROUP BY month
ORDER BY month ASC
"""

result =  pd.read_sql_query(query, conn_1)
print(result)

   month  avg_daily_inches_rain
0     01               0.191613
1     02               0.094277
2     03               0.113578
3     04               0.085364
4     05               0.068035
5     06               0.050182
6     07               0.016129
7     08               0.034370
8     09               0.056930
9     10               0.115543
10    11               0.216242
11    12               0.180059


-------

## 2. Employee

In [31]:
department = pd.read_csv("02_employee/department.csv")
employee = pd.read_csv("02_employee/employee_data.csv")

conn_2 = sqlite3.connect(":memory:")

department.to_sql('department', conn_2, index=False)
employee.to_sql('employee', conn_2, index=False)

16

**1. List all the employees in order of descreasing salary**

In [33]:
query = """
SELECT *
FROM employee
ORDER BY salary DESC
"""

result =  pd.read_sql_query(query, conn_2)
print(result)

    emp_id          emp_name  mananger_id  dept_id  salary
0        4     Anthony Stark          NaN        1  100000
1        8             Shuri          NaN        5   80000
2       15      Joe McMillan         13.0        2   80000
3       13      Donna Clarke          NaN        1   75000
4       14      Cameron Howe         13.0        3   75000
5        9   Diane Rawlinson          NaN        1   50000
6       11    Nancy Karrigan          9.0        4   42000
7       12  Kristi Yamaguchi          9.0        4   40000
8       16      Saul Goodman         13.0        6   40000
9       10     Tonya Harding          9.0        4   30000
10       6      Bruce Banner          4.0        5    8000
11       7       Jane Foster          4.0        5    7500
12       1     Micheal Scott          NaN        1    6000
13       2    Dwight Schrute          1.0        2    4000
14       5          Bill Nye          4.0        5    3800
15       3       Jim Halpert          1.0        2    35

**2. List all the department names, and the number of employees in that department. Order by number of employess in department (greatest to least)**

In [35]:
query = """
SELECT 
    d.deptname,
    d.deptid,
    COUNT(d.deptname)
FROM department AS d
INNER JOIN employee AS e ON d.deptid = e.dept_id
GROUP BY d.deptname
ORDER BY COUNT(d.deptname) DESC

"""

result =  pd.read_sql_query(query, conn_2)
print(result)

   deptname  deptid  COUNT(d.deptname)
0  Research       5                  4
1     Admin       1                  4
2  Training       4                  3
3     Sales       2                  3
4     Legal       6                  1
5        IT       3                  1


**3. List all the employees that don't have a manager**

In [37]:
query = """
SELECT *
FROM employee
WHERE mananger_id IS NULL
"""

result =  pd.read_sql_query(query, conn_2)
print(result)

   emp_id         emp_name mananger_id  dept_id  salary
0       1    Micheal Scott        None        1    6000
1       4    Anthony Stark        None        1  100000
2       8            Shuri        None        5   80000
3       9  Diane Rawlinson        None        1   50000
4      13     Donna Clarke        None        1   75000


**4. List all employees by name, and the name of their manager. If the employee doesn't have a manager, leave the column as NULL.**

In [39]:
query = """
SELECT 
    e1.emp_name AS employee_name,
    e2.emp_name AS manager_name
FROM employee AS e1
LEFT JOIN employee AS e2 ON e1.mananger_id = e2.emp_id
ORDER by manager_name DESC
"""

result =  pd.read_sql_query(query, conn_2)
print(result)

       employee_name     manager_name
0     Dwight Schrute    Micheal Scott
1        Jim Halpert    Micheal Scott
2       Cameron Howe     Donna Clarke
3       Joe McMillan     Donna Clarke
4       Saul Goodman     Donna Clarke
5      Tonya Harding  Diane Rawlinson
6     Nancy Karrigan  Diane Rawlinson
7   Kristi Yamaguchi  Diane Rawlinson
8           Bill Nye    Anthony Stark
9       Bruce Banner    Anthony Stark
10       Jane Foster    Anthony Stark
11     Micheal Scott             None
12     Anthony Stark             None
13             Shuri             None
14   Diane Rawlinson             None
15      Donna Clarke             None


**5. For each manager, list the number of employees he or she is managing. For these purposes, a manager is anyone who is not managed by someone else, even if that person has no direct reports.**

In [41]:
query = """
SELECT
    e1.emp_name AS manager_name,
    COUNT(e2.emp_id) AS num_managed
FROM employee AS e1
LEFT JOIN employee as e2 ON e1.emp_id = e2.mananger_id
WHERE e2.mananger_id IS NOT NULL
GROUP BY e1.emp_id, e1.emp_name
ORDER BY num_managed DESC

"""

result =  pd.read_sql_query(query, conn_2)
print(result)

      manager_name  num_managed
0    Anthony Stark            3
1  Diane Rawlinson            3
2     Donna Clarke            3
3    Micheal Scott            2


**6. Find the two highest paid people per department**

In [43]:
query = """

SELECT 
    dept_id,
    emp_name,
    salary,
    salary_rank
FROM (
    SELECT 
        emp_name,
        dept_id,
        salary,
        RANK() OVER (PARTITION BY dept_id ORDER BY salary DESC) AS salary_rank
    FROM employee
) ranked
WHERE 
    salary_rank <= 2
"""

result =  pd.read_sql_query(query, conn_2)
print(result)


   dept_id          emp_name  salary  salary_rank
0        1     Anthony Stark  100000            1
1        1      Donna Clarke   75000            2
2        2      Joe McMillan   80000            1
3        2    Dwight Schrute    4000            2
4        3      Cameron Howe   75000            1
5        4    Nancy Karrigan   42000            1
6        4  Kristi Yamaguchi   40000            2
7        5             Shuri   80000            1
8        5      Bruce Banner    8000            2
9        6      Saul Goodman   40000            1


------

## 3. Social Network

In [81]:
follows = pd.read_csv("03_social_network/follows.csv")
users = pd.read_csv("03_social_network/users.csv")

conn_3 = sqlite3.connect(":memory:")

follows.to_sql('follows', conn_3, index=False)
users.to_sql('users', conn_3, index=False)

9

**1. How many users are there in each house?**

In [97]:
query = """
SELECT
    house,
    count(house) AS Count
FROM users
GROUP BY house
"""

result =  pd.read_sql_query(query, conn_3)
print(result)


        house  Count
0  Gryffindor      4
1  Hufflepuff      1
2   Ravenclaw      2
3   Slytherin      2


**2. List all following links that were created before September 1st, 1993**

In [110]:
query = """
SELECT *
FROM follows
WHERE date < "1993-09-01"
"""

result =  pd.read_sql_query(query, conn_3)
print(result)

   user_id  follows        date
0        2        1  1989-01-01
1        3        1  1993-07-01
2        4        2  1988-08-08
3        4        1  1988-08-08
4        5        6  1986-01-10
5        7        1  1990-02-02


**3. List all rows from the follows table, replacing both user_ids with first name. Hint: it may help to make this a VIEW**

In [167]:
query = """
CREATE VIEW follow_names AS
SELECT 
    u1.first_name AS follower,
    u2.first_name AS followed,
    f.date
FROM 
    follows f
JOIN 
    users u1 ON f.user_id = u1.user_id
JOIN 
    users u2 ON f.follows = u2.user_id;
"""

result =  pd.read_sql_query(query, conn_3)


TypeError: 'NoneType' object is not iterable

In [171]:
query = """
SELECT * FROM follow_names;
"""

result =  pd.read_sql_query(query, conn_3)
print(result)

    follower  followed        date
0      Harry       Ron  1993-09-01
1        Ron     Harry  1989-01-01
2   Hermonie     Harry  1993-07-01
3        Ron  Hermonie  1994-10-10
4   Hermonie       Ron  1995-03-01
5      Ginny       Ron  1988-08-08
6      Ginny     Harry  1988-08-08
7      Harry     Ginny  1994-04-02
8      Harry     Draco  2000-01-01
9      Draco     Harry  2000-01-02
10     Draco       Tom  1986-01-10
11      Luna     Harry  1990-02-02
12     Harry      Luna  1996-10-01
13     Harry       Cho  1993-09-03
14       Cho     Harry  1995-09-01
15       Cho    Cedric  1995-09-01
16    Cedric       Cho  1996-01-10
17      Luna       Cho  1993-09-01
18  Hermonie    Cedric  1996-05-30
19     Ginny    Cedric  1996-05-30


**4. List all the following links established before September 1st 1993, but this time use the users first names.**

In [174]:
query = """
SELECT * 
FROM follow_names
WHERE date < "1993-09-01"
"""

result =  pd.read_sql_query(query, conn_3)
print(result)

   follower followed        date
0       Ron    Harry  1989-01-01
1  Hermonie    Harry  1993-07-01
2     Ginny      Ron  1988-08-08
3     Ginny    Harry  1988-08-08
4     Draco      Tom  1986-01-10
5      Luna    Harry  1990-02-02


**5. Give a count of how many people followed each user as of 1999-12-31. Give the result in term of "users full name, number of followers".**

In [205]:
query = """
SELECT 
    u.first_name || ' ' || u.last_name AS user_followed_name,
    COUNT(f.follows) AS num_followers
FROM users AS u
JOIN follows AS f ON u.user_id = f.follows
WHERE date <= "1999-12-31"
GROUP BY user_followed_name
ORDER BY u.user_id

"""

result =  pd.read_sql_query(query, conn_3)
print(result)

  user_followed_name  num_followers
0       Harry Potter              5
1         Ron Wesley              3
2   Hermonie Granger              1
3      Ginny Weasley              1
4         Tom Riddle              1
5      Luna Lovegood              1
6          Cho Chang              3
7     Cedric Diggory              3


**6. List the number of users each user follows**

In [214]:
query = """
SELECT 
    u.first_name || ' ' || u.last_name AS user_followed_name,
    COUNT(f.user_id) AS num_followed
FROM users AS u
JOIN follows AS f ON u.user_id = f.user_id
GROUP BY user_followed_name
ORDER BY u.user_id

"""

result =  pd.read_sql_query(query, conn_3)
print(result)

  user_followed_name  num_followed
0       Harry Potter             5
1         Ron Wesley             2
2   Hermonie Granger             3
3      Ginny Weasley             3
4       Draco Malfoy             2
5      Luna Lovegood             2
6          Cho Chang             2
7     Cedric Diggory             1


**7. List all rows from follows where someone from one house follows someone from a different house.**

In [277]:
query = """
SELECT 
    u.first_name || ' ' || u.last_name AS username,
    u.house AS user_house,
    u2.first_name || ' ' || u2.last_name AS follow_name,
    u2.house AS follows_house
FROM users AS u
JOIN follows AS f ON u.user_id = f.user_id
JOIN users AS u2 ON f.follows = u2.user_id
WHERE u.house != u2.house
"""

result =  pd.read_sql_query(query, conn_3)
print(result)

           username  user_house     follow_name follows_house
0      Harry Potter  Gryffindor    Draco Malfoy     Slytherin
1      Harry Potter  Gryffindor   Luna Lovegood     Ravenclaw
2      Harry Potter  Gryffindor       Cho Chang     Ravenclaw
3  Hermonie Granger  Gryffindor  Cedric Diggory    Hufflepuff
4     Ginny Weasley  Gryffindor  Cedric Diggory    Hufflepuff
5      Draco Malfoy   Slytherin    Harry Potter    Gryffindor
6     Luna Lovegood   Ravenclaw    Harry Potter    Gryffindor
7         Cho Chang   Ravenclaw    Harry Potter    Gryffindor
8         Cho Chang   Ravenclaw  Cedric Diggory    Hufflepuff
9    Cedric Diggory  Hufflepuff       Cho Chang     Ravenclaw


**8. We define a friendship as a relationship between two users where both follow each other. The friendship is established when the later of the two links is established.**

In [304]:
query = """
SELECT 
    f1.user_id AS user1_id,
    f2.user_id AS user2_id,
    MAX(f1.date, f2.date) AS friendship_date
FROM 
    follows f1
JOIN 
    follows f2
    ON f1.user_id = f2.follows AND f1.follows = f2.user_id
WHERE 
    f1.user_id < f2.user_id

"""

result =  pd.read_sql_query(query, conn_3)
print(result)

   user1_id  user2_id friendship_date
0         1         2      1993-09-01
1         2         3      1995-03-01
2         1         4      1994-04-02
3         1         5      2000-01-02
4         1         7      1996-10-01
5         1         8      1995-09-01
6         8         9      1996-01-10


In [300]:
query = """
SELECT 
    u1.first_name || ' ' || u1.last_name AS user1_id,
    u2.first_name || ' ' || u2.last_name AS user1_id,
    MAX(f1.date, f2.date) AS friendship_date
FROM 
    follows f1
JOIN 
    follows f2
    ON f1.user_id = f2.follows AND f1.follows = f2.user_id
JOIN
    users u1
    ON f1.user_id = u1.user_id
JOIN
    users u2
    ON f2.user_id = u2.user_id
WHERE 
    f1.user_id < f2.user_id

"""

result =  pd.read_sql_query(query, conn_3)
print(result)

       user1_id          user1_id friendship_date
0  Harry Potter        Ron Wesley      1993-09-01
1    Ron Wesley  Hermonie Granger      1995-03-01
2  Harry Potter     Ginny Weasley      1994-04-02
3  Harry Potter      Draco Malfoy      2000-01-02
4  Harry Potter     Luna Lovegood      1996-10-01
5  Harry Potter         Cho Chang      1995-09-01
6     Cho Chang    Cedric Diggory      1996-01-10


**9. List all unrequited followings (i.e. where A follows B but B does not follow A)**

In [403]:
query = """
SELECT 
    f1.user_id AS follower,
    u1.first_name || ' ' || u1.last_name AS follower_name,
    f1.follows AS followed,
    u2.first_name || ' ' || u2.last_name AS followed_name
FROM 
    follows f1
JOIN
    users u1
    ON f1.user_id = u1.user_id
JOIN
    users u2
    ON f1.follows = u2.user_id
LEFT JOIN 
    follows f2
    ON f1.user_id = f2.follows AND f1.follows = f2.user_id
WHERE 
    f2.user_id IS NULL

"""

result =  pd.read_sql_query(query, conn_3)
print(result)

   follower     follower_name  followed   followed_name
0         3  Hermonie Granger         1    Harry Potter
1         4     Ginny Weasley         2      Ron Wesley
2         5      Draco Malfoy         6      Tom Riddle
3         7     Luna Lovegood         8       Cho Chang
4         3  Hermonie Granger         9  Cedric Diggory
5         4     Ginny Weasley         9  Cedric Diggory
